# Signal Reconstruction

[Mauricio de Oliveira](http://control.ucsd.edu/mauricio)

*September, 2019*

Under certain conditions (see notebook on Aliasing) it is possible to *exactly* reconstruct a continuous signal from its samples. This is what you will explore in this notebook.

## Load some libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import signal
import librosa
import IPython.display as ipd

Generate a sinusoidal tone at f1 = 3Hz, x1, with a duration of T = 10s sampled at fs1 = 1000Hz sampling rate. This sampling rate is significantly higher than the frequency of the tone, which will make it appear smooth. Use $\cos$ to generate the waveform.

In [ ]:
T = 10.0

fs1 = 1000
t_1000 = np.arange(0, T, 1/fs1)

f1 = 3.0
A1 = .8
phi1 = np.pi*35/180

x1_1000 = A1*np.cos(2*np.pi*f1*t_1000+phi1)

Repeat with fs2 = 10Hz as the sampling rate and compare the two sets of samples. This choice is such that f1 = 3Hz is below the Nyquist frequency (see notebook on Aliasing)

In [ ]:
fs2 = 10
t_10 = np.arange(0, T, 1/fs2)

x1_10 = A1*np.cos(2*np.pi*f1*t_10+phi1)

plt.plot(t_1000, x1_1000, '-', t_10, x1_10, 'yo')
plt.title('Sine Waves, f1 = {0}Hz, Sampling rates = {1}Hz and {2}Hz'.format(f1,fs1,fs2));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

Plot only the samples at fs2 = 10Hz to see how they *do not* immediately suggest this is a sinusoidal signal!

In [ ]:
plt.plot(t_10, x1_10, 'yo')
plt.title('Sine Waves, f1 = {0}Hz, Sampling rate = {1}Hz'.format(f1,fs2));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

In order to *exactly* reconstruct the continuos signal, $x(t)$, from its samples, $x(k T_s)$, where $T_s$ is the sampling period and $k$ is an integer, we can use the following formula:

<center>
$x(t) = \sum_{k = -\infty}^{\infty} x(k T_s) \operatorname{sinc}(t/Ts - k), \qquad \operatorname{sinc}(x) = \displaystyle \frac{\sin{\pi x}}{\pi x},$
</center>

known as *Shannon interpolation formula*.

There are two catches: first the formula adds an infinite number of samples, and second the continuous signal $\operatorname{sinc}(t)$ is of infinite duration. If we use a finite ammount of samples and evaluate the signal $\operatorname{sinc}$ on a finite interval we shall recover the function only approximatelly. In practice this is enough.

This is implemented next as a for loop, where we use the higher sampling rate to emulate the continuous signal $\operatorname{sinc}$:

In [ ]:
Ts2 = 1./fs2
x1r_1000 = np.zeros(t_1000.shape)
for k in range(len(x1_10)):
    x1r_1000 = x1r_1000 + x1_10[k] * np.sinc((t_1000-k*Ts2)/Ts2)

We compare the *original* signal with the one reconstructed:

In [ ]:
plt.plot(t_1000, x1_1000, '-', t_1000, x1r_1000, '-')
plt.title('Sampled Sine Waves, f1 = {0} Hz, and alias, Sampling rate {1} Hz'.format(f1,fs1));
plt.xlabel('t (s)');
plt.ylabel('x(t), xr(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

Not too bad, with a bit of distortion at the edge.

## Rich Signals

The reconstruction process for signals that are rich in higher harmonics is the same as in the case of a single tone. We illustrate it next for a sawtooth waveform. The distortions that occur in the reconstructed signal are not due to the reconstruction but due to alliasing (see Aliasing notebook).

In [ ]:
N = 2**16
f2 = 2
x2_1000 = 2*((N*(.75-f2*t_1000) % N)/N - .5)
x2_10 = 2*((N*(.75-f2*t_10) % N)/N - .5)

plt.plot(t_1000, x2_1000, '-', t_10, x2_10, 'o')
plt.title('Sawtooth Wave, f1 = {0} Hz, Sampling rate = {1}Hz'.format(f2,fs2));
plt.xlabel('t (s)');
plt.ylabel('x(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

In [ ]:
x2r_1000 = np.zeros(t_1000.shape)
for k in range(len(t_10)):
    x2r_1000 = x2r_1000 + x2_10[k] * np.sinc(fs2*t_1000-k)

In [ ]:
plt.plot(t_1000, x2_1000, '-', t_1000, x2r_1000, '-')
plt.title('Reconstructed Sawtooth Wave, f1 = {0} Hz, Sampling rate {1} Hz'.format(f2,fs2));
plt.xlabel('t (s)');
plt.ylabel('x(t), xr(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

In ths case of rich signals the reconstruction is not as good. Besides the issue with the finite approximation of the Shannon formula, the rich signal violates a key assumption required for exact reconstruction: that the original signal be bandlimited. The triangular wave has an infinite number of harmonics.

## Zero-order holder reconstruction

In practice, simpler interpolation schemes are used for signal reconstruction.

A popular scheme is that of a zero-order-holder followed by a low-pass filter.

The zero-order interpolation of the sawtooth signal can be constructed as follows:

In [ ]:
tt = np.vstack((t_10,t_10)).T.flatten()[1:]
xx = np.vstack((x2_10,x2_10)).T.flatten()[:-1]
x2r_1000 = np.interp(t_1000, tt, xx)

In [ ]:
plt.plot(t_1000, x2_1000, '-', t_1000, x2r_1000, '-')
plt.title('Reconstructed Sawtooth Wave, f1 = {0} Hz, Sampling rate {1} Hz'.format(f2,fs2));
plt.xlabel('t (s)');
plt.ylabel('x(t), xr(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

The resulting signal can be made more smooth using a low-pass filter:

In [ ]:
fc = 4  # Cut-off frequency of the filter
w = fc / (fs1 / 2) # Normalize the frequency
b, a = signal.butter(5, w, 'low')
x2rf_1000 = signal.filtfilt(b, a, x2r_1000)

In [ ]:
plt.plot(t_1000, x2_1000, '-', t_1000, x2r_1000, '-', t_1000, x2rf_1000, '-')
plt.title('Reconstructed Sawtooth Wave, f1 = {0} Hz, Sampling rate {1} Hz'.format(f2,fs2));
plt.xlabel('t (s)');
plt.ylabel('x(t), xr(t)');
plt.xlim(0,2)
plt.grid(1)
plt.show()

Let's us repeat this exercise with a music file.

Upload your favorite song by dragging it and dropping on the notebook file manager, then change the name below to reflect the filename of the song. We will only load 20s of the song to make things faster. We will also work with a mono version of the song for the same reason.

In [ ]:
x1, fs = librosa.load('Stevie Wonder Superstition.mp3', mono=True, offset=20, duration=20)

In [ ]:
plt.plot(x1[11*fs:11*fs+round(.005* fs)], '-')
plt.title('Sampling Rate = {0}Hz, Mono'.format(fs));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.grid(1)
plt.show()

Play the song

In [ ]:
ipd.Audio(x1, rate=fs)

We will now process this song by first subsampling the signal and then reconstructing the signal with a zero-order holder.

Before subsampling we apply a low-pass filter to reduce the effects of aliasing.

Subsampling will be done at the frequency `fss` so a low-pass filter will be aplied with a cut-off frequency of `fss/2`.

In [ ]:
N = 7
fss = fs / N
fc = fss / 2  # Cut-off frequency of the filter
w = fc / (fs / 2) # Normalize the frequency
b, a = signal.butter(5, w, 'low')
x1f = signal.filtfilt(b, a, x1)

Note how the filtered signal is *smoother* than the original signal:

In [ ]:
dt = 0.008
plt.plot(x1[11*fs:11*fs+round(dt*fs)], '-', x1f[11*fs:11*fs+round(dt*fs)], '-')
plt.title('Sampling Rate = {0}Hz, Original and filtered'.format(fs));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.grid(1)
plt.show()

Now play the filtered signal. This is what the signal we will reconstruct later will be compared against. 

In [ ]:
ipd.Audio(x1f, rate=fs)

We will now subsample the filtered signal

In [ ]:
x1ss = x1f[::N]

and reconstruct it using a zero-order holder as done above

In [ ]:
T = len(x1) / fs
tss = np.arange(0, T, 1/fss)
t = np.arange(0, T, 1/fs)
tt = np.vstack((tss,tss)).T.flatten()[1:]
xx = np.vstack((x1ss,x1ss)).T.flatten()[:-1]
x1zoh = np.interp(t, tt, xx)

Compare the filtered signal with its sampled and zero-order hold version

In [ ]:
dt = 0.008
plt.plot(x1f[11*fs:11*fs+round(dt*fs)], '-', x1zoh[11*fs:11*fs+round(dt*fs)], '-')
plt.title('Sampling Rate = {0}Hz, Filtered and ZOH'.format(fs));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.grid(1)
plt.show()

Playing the zero-order hold reconstruction directly results in a very rough sound because of the high-frequencies introduced by the discontinuities:

In [ ]:
ipd.Audio(x1zoh, rate=fs)

Those are all but gone after filtering:

In [ ]:
x1zohf = signal.filtfilt(b, a, x1zoh)

See how close the two signals are? Can you make sense of the small delay?

In [ ]:
dt = 0.008
plt.plot(x1f[11*fs:11*fs+round(dt*fs)], '-', x1zohf[11*fs:11*fs+round(dt*fs)], '-')
plt.title('Sampling Rate = {0}Hz, Filtered and ZOH+filter'.format(fs));
plt.xlabel('t (s)');
plt.ylabel('x1(t)');
plt.grid(1)
plt.show()

Play it for further delight:

In [ ]:
ipd.Audio(x1zohf, rate=fs)